Simulations and estimators examples

All ideal boosted simulations used here were generated with [Healpix Boost](https://github.com/mquartin/healpix-boost).

# Importing Modules

In [ ]:
import os                                                                     
from threadpoolctl import threadpool_limits
import healpy as hp
import numpy as np
from tqdm import tqdm
from astropy.io import fits as pyfits
import random
import pymp
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
import importlib
import sys
import multiprocessing as mp
from math import sqrt
from math import radians
import gc
import subprocess
gc.disable()
##################################################################################################################
from cmbaberdopp_beta0p8 import * # cmbaberdopp_beta0p8 its a module created w/ the functions needed for this work
                                  # remember to change the path of maps on dipole_doppler_load_low_res function    
##################################################################################################################
def c_nthreads_limit(n): # Function to change number of threads used on imported C/C++ functions
    threadpool_limits(limits=n, user_api='blas')
    threadpool_limits(limits=n, user_api='openmp')


# SIMULATIONS EXAMPLES


## Applying NILC DD on ideal TT sims (MP)

In [ ]:
##############
#### NILC ####
##############
ideal_doppler_sims_dir = '/ideal_doppler/'
ideal_ab_sims_dir = '/ideal_aberration/'
ideal_abdopp_sims_dir = '/ideal_abdopp/'

nilc_dd_doppler_sims_dir = '/nilc_dd_dopp_sims/'
nilc_dd_ab_sims_dir = '/nilc_dd_ab_sims/'
nilc_dd_abdopp_sims_dir = '/nilc_dd_abdopp_sims/'

nside_rot = 2 # setting the directions do you want to apply the modulations
nsims_per_dir = 64

alm=hp.read_alm(ideal_doppler_sims_dir+'doppler_boosted_alm_dir_'+str(1)+'_sim_'+str(1)+'.fits') # used to gett structure of file

# Loading NILC Dipole Distortion modulation maps - cache
dipole_doppler_load_low_res(alm=alm,pipeline='nilc',almfile=False,lmax=2048,overbin=10,threads=16,dd_dir="/dd_maps/",verbose=True)

nthreads = 2
c_nthreads_limit(nthreads)
nprocesses = 6
for n in range(hp.nside2npix(nside_rot)):
    for i in range(1,nsims_per_dir+1):
        print(n,i)
        start = time.time()
        
        #applying DD on doppler ideal simulations
        doppler_alm = hp.read_alm(ideal_doppler_sims_dir+'doppler_boosted_alm_dir_'+str(n+1)+'_sim_'+str(i)+'.fits')
        nilc_dd_alm = dipole_doppler_low_res_weights(doppler_alm,threads=nthreads,processes=nprocesses)
        hp.write_alm(nilc_dd_doppler_sims_dir+'doppler_boosted_alm_w_nilc_dd_dir_'+str(n+1)+'_sim_'+str(i)+'.fits',nilc_dd_alm,overwrite=True)

        #applying DD on aberration ideal simulations
        ab_alm = hp.read_alm(ideal_ab_sims_dir+'rot_ab_boosted_dir_'+str(n+1)+'_alm_'+str(i)+'.fits')
        nilc_dd_alm = dipole_doppler_low_res_weights(ab_alm,threads=nthreads,processes=nprocesses)
        hp.write_alm(nilc_dd_ab_sims_dir+'ab_boosted_alm_w_nilc_dd_dir_'+str(n+1)+'_sim_'+str(i)+'.fits',nilc_dd_alm,overwrite=True)

        #applying DD on boost ideal simulations
        abdopp_alm = hp.read_alm(ideal_abdopp_sims_dir+'abdopp_boosted_alm_dir_'+str(n+1)+'_sim_'+str(i)+'.fits')
        nilc_dd_alm = dipole_doppler_low_res_weights(abdopp_alm,threads=nthreads,processes=nprocesses)
        hp.write_alm(nilc_dd_abdopp_sims_dir+'abdopp_boosted_alm_w_nilc_dd_dir_'+str(n+1)+'_sim_'+str(i)+'.fits',nilc_dd_alm,overwrite=True)
    
        end = time.time()
        print(end-start)


## Applying SMICA DD on ideal sims (MP) - Maybe will need to restart the kernel before run

In [ ]:
##############
#### SMICA ###
## DOPPLER ###
##############
ideal_doppler_sims_dir = '/ideal_doppler/'
ideal_ab_sims_dir = '/ideal_aberration/'
ideal_abdopp_sims_dir = '/ideal_abdopp/'

smica_dd_doppler_sims_dir = '/smica_dd_dopp_sims/'
smica_dd_ab_sims_dir = '/smica_dd_ab_sims/'
smica_dd_abdopp_sims_dir = '/smica_abdioo_dopp_sims/'

nside_rot = 2 # setting the directions do you want to apply the modulations
nsims_per_dir = 64

alm=hp.read_alm('doppler_boosted_alm_dir_'+str(1)+'_sim_'+str(1)+'.fits') # used to gett structure of file
dipole_doppler_load_low_res(alm=alm,pipeline='smica',almfile=False,lmax=2048,overbin=10,threads=16,dd_dir="",verbose=True)

nthreads = 2
c_nthreads_limit(nthreads)
nprocesses = 6
for n in range(hp.nside2npix(nside_rot)):
    for i in range(1,nsims_per_dir+1):
        print(n,i)
        start = time.time()
        
        #applying DD on doppler ideal simulations
        doppler_alm = hp.read_alm(ideal_doppler_sims_dir+'doppler_boosted_alm_dir_'+str(n+1)+'_sim_'+str(i)+'.fits')
        nilc_dd_alm = dipole_doppler_low_res_weights(doppler_alm,threads=nthreads,processes=nprocesses)
        hp.write_alm('doppler_boosted_alm_w_smica_dd_dir_'+str(n+1)+'_sim_'+str(i)+'.fits',nilc_dd_alm,overwrite=True)

        #applying DD on aberration ideal simulations
        ab_alm = hp.read_alm(ideal_ab_sims_dir+'rot_ab_boosted_dir_'+str(n+1)+'_alm_'+str(sim)+'.fits')
        nilc_dd_alm = dipole_doppler_low_res_weights(ab_alm,threads=nthreads,processes=nprocesses)
        hp.write_alm('ab_boosted_alm_w_smica_dd_dir_'+str(n+1)+'_sim_'+str(sim)+'.fits',nilc_dd_alm,overwrite=True)

        #applying DD on boost ideal simulations
        abdopp_alm = hp.read_alm(ideal_abdopp_sims_dir+'abdopp_boosted_alm_dir_'+str(n+1)+'_sim_'+str(sim)+'.fits')
        nilc_dd_alm = dipole_doppler_low_res_weights(abdopp_alm,threads=nthreads,processes=nprocesses)
        hp.write_alm('abdopp_boosted_alm_w_smica_dd_dir_'+str(n+1)+'_sim_'+str(sim)+'.fits',nilc_dd_alm,overwrite=True)
    
        end = time.time()
        print(end-start)

## Applying mask and noise (MP)

### NILC (MP)

In [ ]:
##########################################
## Applying Mask and Deconvolved Noise ###
##########################################
################# NILC ###################
##########################################
mask_files_dir = '/mask_files/'
noise_files_dir = '/noise_T_E_B/'
nilc_dopp_dd_sims_dir = '/nilc_dd_dopp_sims/'
nilc_ab_dd_sims_dir = '/nilc_dd_ab_sims/'
nilc_abdopp_dd_sims_dir = '/nilc_dd_abdopp_sims/'
nilc_dd_w_noise_sims_ab_sims = '/nilc_dd_ab_w_noise_sims/'
nilc_dd_w_noise_sims_dopp_sims = '/nilc_dd_dopp_w_noise_sims/'
nilc_dd_w_noise_sims_abdopp_sims = '/nilc_dd_abdopp_w_noise_sims/'

nside_rot = 2 # setting the directions do you want to apply the modulations
nsims_per_dir = 64

nthreads= 4
c_nthreads_limit(nthreads)
nprocesses = 3

symmmask = hp.read_map(str(mask_files_dir)+'UT79symmapod10.fits',verbose=False) # antipodal symmetric mask
def noise_nilc_loop(n):
    start_dir = time.time()
    for i in range(nsims_per_dir):
        start_sim = time.time()
        print(n,i+1)
        noise_alm = hp.read_alm(str(noise_files_dir)+'debeamed_nilc_T_noise_'+str(np.random.randint(300))+'.fits')
        noise = hp.alm2map(noise_alm,nside=2048)
        
        #importing alms w DD
        doppler_alm_dd = hp.read_alm(str(nilc_dopp_dd_sims_dir)+'doppler_boosted_alm_w_nilc_dd_dir_'+str(n+1)+'_sim_'+str(i+1)+'.fits')
        aberration_alm_dd = hp.read_alm(str(nilc_ab_dd_sims_dir)+'ab_boosted_alm_w_nilc_dd_dir_'+str(n+1)+'_sim_'+str(i+1)+'.fits')        
        abdopp_alm_dd = hp.read_alm(str(nilc_abdopp_dd_sims_dir)+'abdopp_boosted_alm_w_nilc_dd_dir_'+str(n+1)+'_sim_'+str(i+1)+'.fits')        
        
        #generating maps and adding noise
        doppler_noise_map_dd = hp.alm2map(doppler_alm_dd,nside=2048,verbose=False)+noise
        aberration_noise_map_dd = hp.alm2map(aberration_alm_dd,nside=2048,verbose=False)+noise
        abdopp_noise_map_dd = hp.alm2map(abdopp_alm_dd,nside=2048,verbose=False)+noise
        
        #applying symmetric mask
        doppler_noise_symmmasked_map_dd = doppler_noise_map_dd*symmmask
        aberration_noise_symmmasked_map_dd = aberration_noise_map_dd*symmmask
        abdopp_noise_symmmasked_map_dd = abdopp_noise_map_dd*symmmask
                
        #generating symmmasked alms
        doppler_noise_symmmasked_alm_dd = hp.map2alm(doppler_noise_symmmasked_map_dd,lmax=2048,iter=1)
        aberration_noise_symmmasked_alm_dd = hp.map2alm(aberration_noise_symmmasked_map_dd,lmax=2048,iter=1)
        abdopp_noise_symmmasked_alm_dd = hp.map2alm(abdopp_noise_symmmasked_map_dd,lmax=2048,iter=1)
        
        #saving alms
        hp.write_alm(str(nilc_dd_w_noise_sims_dopp_sims)+'doppler_boosted_alm_symmmasked_w_nilc_dd_and_noise_beam_deconvolved_dir_'+str(n+1)+'_sim_'+str(i+1)+'.fits',doppler_noise_symmmasked_alm_dd,overwrite=True)
        hp.write_alm(str(nilc_dd_w_noise_sims_ab_sims)+'ab_boosted_alm_symmmasked_w_nilc_dd_and_noise_beam_deconvolved_dir_'+str(n+1)+'_sim_'+str(i+1)+'.fits',aberration_noise_symmmasked_alm_dd,overwrite=True)
        hp.write_alm(str(nilc_dd_w_noise_sims_abdopp_sims)+'abdopp_boosted_alm_symmmasked_w_nilc_dd_and_noise_beam_deconvolved_dir_'+str(n+1)+'_sim_'+str(i+1)+'.fits',abdopp_noise_symmmasked_alm_dd,overwrite=True)
        
        end_sim = time.time()
        print(end_sim-start_sim)
        
    end_dir = time.time()
    print(end_dir-start_dir)
        
pool = mp.Pool(processes=nprocesses)
pool.map(noise_nilc_loop, range(hp.nside2npix(nside_rot)))
pool.terminate()
pool.join()

### SMICA (MP)

In [ ]:
##########################################
## Applying Mask and Deconvolved Noise ###
##########################################
################# SMICA ##################
##########################################

mask_files_dir = '/mask_files/'
noise_files_dir = '/noise_T_E_B/'
smica_dopp_dd_sims_dir = '/smica_dd_dopp_sims/'
smica_ab_dd_sims_dir = '/smica_dd_ab_sims/'
smica_abdopp_dd_sims_dir = '/smica_dd_abdopp_sims/'
smica_dd_w_noise_sims_ab_sims = '/smica_dd_ab_w_noise_sims/'
smica_dd_w_noise_sims_dopp_sims = '/smica_dd_dopp_w_noise_sims/'
smica_dd_w_noise_sims_abdopp_sims = '/smica_dd_abdopp_w_noise_sims/'

nside_rot = 2 # setting the directions do you want to apply the modulations
nsims_per_dir = 64

nthreads= 2
c_nthreads_limit(nthreads)
nprocesses = 5

symmmask = hp.read_map(str(mask_files_dir)+'UT79symmapod10.fits',verbose=False)# antipodal symmetric mask
def noise_smica_loop(n):
    start_dir = time.time()
    for i in range(nsims_per_dir):
        start_sim = time.time()
        print(n,i+1)
        noise_alm = hp.read_alm(str(noise_files_dir)+'debeamed_smica_noise_'+str(np.random.randint(300))+'.fits')
        noise = hp.alm2map(noise_alm,nside=2048)
        
        #importing alms w DD
        doppler_alm_dd = hp.read_alm(str(smica_dopp_dd_sims_dir)+'doppler_boosted_alm_w_smica_dd_dir_'+str(n+1)+'_sim_'+str(sim)+'.fits')
        aberration_alm_dd = hp.read_alm(str(smica_ab_dd_sims_dir)+'ab_boosted_alm_w_smica_dd_dir_'+str(n+1)+'_sim_'+str(sim)+'.fits')        
        abdopp_alm_dd = hp.read_alm(str(smica_abdopp_dd_sims_dir)+'abdopp_boosted_alm_w_smica_dd_dir_'+str(n+1)+'_sim_'+str(sim)+'.fits')        
        
        #generating maps and adding noise
        doppler_noise_map_dd = hp.alm2map(doppler_alm_dd,nside=2048,verbose=False)+noise
        aberration_noise_map_dd = hp.alm2map(aberration_alm_dd,nside=2048,verbose=False)+noise
        abdopp_noise_map_dd = hp.alm2map(abdopp_alm_dd,nside=2048,verbose=False)+noise
        
        #applying symm mask
        doppler_noise_symmmasked_map_dd = doppler_noise_map_dd*symmmask
        aberration_noise_symmmasked_map_dd = aberration_noise_map_dd*symmmask
        abdopp_noise_symmmasked_map_dd = abdopp_noise_map_dd*symmmask
                
        #generating symmmasked alms
        doppler_noise_symmmasked_alm_dd = hp.map2alm(doppler_noise_symmmasked_map_dd,lmax=2048,iter=1)
        aberration_noise_symmmasked_alm_dd = hp.map2alm(aberration_noise_symmmasked_map_dd,lmax=2048,iter=1)
        abdopp_noise_symmmasked_alm_dd = hp.map2alm(abdopp_noise_symmmasked_map_dd,lmax=2048,iter=1)
        
        #saving alms
        hp.write_alm(str(smica_dd_w_noise_dopp_sims)+'doppler_boosted_alm_symmmasked_w_smica_dd_and_noise_beam_deconvolved_dir_'+str(n+1)+'_sim_'+str(i+1)+'.fits',doppler_noise_symmmasked_alm_dd,overwrite=True)
        hp.write_alm(str(smica_dd_w_noise_ab_sims)+'ab_boosted_alm_symmmasked_w_smica_dd_and_noise_beam_deconvolved_dir_'+str(n+1)+'_sim_'+str(i+1)+'.fits',aberration_noise_symmmasked_alm_dd,overwrite=True)
        hp.write_alm(str(smica_dd_w_noise_abdopp_sims)+'abdopp_boosted_alm_symmmasked_w_smica_dd_and_noise_beam_deconvolved_dir_'+str(n+1)+'_sim_'+str(i+1)+'.fits',abdopp_noise_symmmasked_alm_dd,overwrite=True)
        
        end_sim = time.time()
        print(end_sim-start_sim)
        
    end_dir = time.time()
    print(end_dir-start_dir)
        
pool = mp.Pool(processes=nprocesses)
pool.map(noise_smica_loop, range(hp.nside2npix(nside_rot)))
pool.terminate()
pool.join()

## Applying mask and deDopplered noise (CCP)

### NILC (CCP)

In [ ]:
###########################################
# Applying Mask and dDD Deconvolved Noise #
###########################################
################# NILC ####################
###########################################

mask_files_dir = ''
noise_files_dir = ''
nilc_ab_dd_sims_dir = ''
nilc_abdopp_dd_sims_dir = ''
nilc_dopp_dd_sims_dir = ''
nilc_dedopplered_noise_ab_sims = '' 
nilc_dedopplered_noise_dopp_sims = ''
nilc_dedopplered_noise_abdopp_sims = '' #SSD

nside_rot = 2 # setting the directions do you want to apply the modulations
nsims_per_dir = 64

nthreads= 2
c_nthreads_limit(nthreads)
nprocesses = 5

symmmask = hp.read_map(str(mask_files_dir)+'UT79symmapod10.fits',verbose=False)
def noise_nilc_loop(n):
    start_dir = time.time()
    for i in range(nsims_per_dir):
        start_sim = time.time()
        print(n,i+1)
        sim = sim2dir_array[n,i]
        noise_alm = hp.read_alm(str(noise_files_dir)+'nilc_debeamed_dedopplered_noise_sim_'+str(np.random.randint(300))+'.fits')
        noise = hp.alm2map(noise_alm,nside=2048)
        
        #importing alms w DD
        doppler_alm_dd = hp.read_alm('doppler_boosted_alm_dir_'+str(n+1)+'_sim_'+str(sim)+'.fits')
        aberration_alm_dd = hp.read_alm('/home/hdd1/cmb_boost/new_ab_sims_2/rot_ab_boosted_dir_'+str(n+1)+'_alm_'+str(sim)+'.fits')        
        abdopp_alm_dd = hp.read_alm('/home/hdd1/cmb_boost/new_gaussian_alms/abdopp_boosted_alm_dir_'+str(n+1)+'_sim_'+str(sim)+'.fits')        
        
        #generating maps and adding noise
        doppler_noise_map_dd = hp.alm2map(doppler_alm_dd,nside=2048,verbose=False)+noise
        aberration_noise_map_dd = hp.alm2map(aberration_alm_dd,nside=2048,verbose=False)+noise
        abdopp_noise_map_dd = hp.alm2map(abdopp_alm_dd,nside=2048,verbose=False)+noise
        
        #applying symm mask
        doppler_noise_symmmasked_map_dd = doppler_noise_map_dd*symmmask
        aberration_noise_symmmasked_map_dd = aberration_noise_map_dd*symmmask
        abdopp_noise_symmmasked_map_dd = abdopp_noise_map_dd*symmmask
                
        #generating symmmasked alms
        doppler_noise_symmmasked_alm_dd = hp.map2alm(doppler_noise_symmmasked_map_dd,lmax=2048,iter=1)
        aberration_noise_symmmasked_alm_dd = hp.map2alm(aberration_noise_symmmasked_map_dd,lmax=2048,iter=1)
        abdopp_noise_symmmasked_alm_dd = hp.map2alm(abdopp_noise_symmmasked_map_dd,lmax=2048,iter=1)
        
        #saving alms
        hp.write_alm(str(nilc_dedopplered_noise_dopp_sims)+'doppler_boosted_alm_symmmasked_w_nilc_dDD_noise_beam_deconvolved_dir_'+str(n+1)+'_sim_'+str(i+1)+'.fits',doppler_noise_symmmasked_alm_dd,overwrite=True)
        hp.write_alm(str(nilc_dedopplered_noise_ab_sims)+'ab_boosted_alm_symmmasked_w_nilc_dDD_noise_beam_deconvolved_dir_'+str(n+1)+'_sim_'+str(i+1)+'.fits',aberration_noise_symmmasked_alm_dd,overwrite=True)
        hp.write_alm(str(nilc_dedopplered_noise_abdopp_sims)+'abdopp_boosted_alm_symmmasked_w_nilc_dDD_noise_beam_deconvolved_dir_'+str(n+1)+'_sim_'+str(i+1)+'.fits',abdopp_noise_symmmasked_alm_dd,overwrite=True)
        
        end_sim = time.time()
        print(end_sim-start_sim)
        
    end_dir = time.time()
    print(end_dir-start_dir)
        
pool = mp.Pool(processes=nprocesses)
pool.map(noise_nilc_loop, range(hp.nside2npix(nside_rot)))
pool.terminate()
pool.join()

### SMICA (CCP)

In [ ]:
###########################################
# Applying Mask and dDD Deconvolved Noise #
###########################################
################# SMICA ###################
###########################################

mask_files_dir = ''
noise_files_dir = ''
smica_ab_dd_sims_dir = ''
smica_abdopp_dd_sims_dir = ''
smica_dopp_dd_sims_dir = ''
smica_dedopplered_noise_sims_ab_sims = '' 
smica_dedopplered_noise_dopp_sims = ''
smica_dedopplered_noise_abdopp_sims = '' #SSD

nside_rot = 2 # setting the directions do you want to apply the modulations
nsims_per_dir = 64

nthreads= 2
c_nthreads_limit(nthreads)
nprocesses = 5

symmmask = hp.read_map(str(mask_files_dir)+'UT79symmapod10.fits',verbose=False)
def noise_nilc_loop(n):
    start_dir = time.time()
    for i in range(nsims_per_dir):
        start_sim = time.time()
        print(n,i+1)
        sim = sim2dir_array[n,i]
        noise_alm = hp.read_alm(str(noise_files_dir)+'smica_debeamed_dedopplered_noise_sim_'+str(np.random.randint(300))+'.fits')
        noise = hp.alm2map(noise_alm,nside=2048)
        
        #importing alms w DD
        doppler_alm_dd = hp.read_alm('doppler_boosted_alm_dir_'+str(n+1)+'_sim_'+str(sim)+'.fits')
        aberration_alm_dd = hp.read_alm('/home/hdd1/cmb_boost/new_ab_sims_2/rot_ab_boosted_dir_'+str(n+1)+'_alm_'+str(sim)+'.fits')        
        abdopp_alm_dd = hp.read_alm('/home/hdd1/cmb_boost/new_gaussian_alms/abdopp_boosted_alm_dir_'+str(n+1)+'_sim_'+str(sim)+'.fits')        
        
        #generating maps and adding noise
        doppler_noise_map_dd = hp.alm2map(doppler_alm_dd,nside=2048,verbose=False)+noise
        aberration_noise_map_dd = hp.alm2map(aberration_alm_dd,nside=2048,verbose=False)+noise
        abdopp_noise_map_dd = hp.alm2map(abdopp_alm_dd,nside=2048,verbose=False)+noise
        
        #applying symm mask
        doppler_noise_symmmasked_map_dd = doppler_noise_map_dd*symmmask
        aberration_noise_symmmasked_map_dd = aberration_noise_map_dd*symmmask
        abdopp_noise_symmmasked_map_dd = abdopp_noise_map_dd*symmmask
                
        #generating symmmasked alms
        doppler_noise_symmmasked_alm_dd = hp.map2alm(doppler_noise_symmmasked_map_dd,lmax=2048,iter=1)
        aberration_noise_symmmasked_alm_dd = hp.map2alm(aberration_noise_symmmasked_map_dd,lmax=2048,iter=1)
        abdopp_noise_symmmasked_alm_dd = hp.map2alm(abdopp_noise_symmmasked_map_dd,lmax=2048,iter=1)
        
        #saving alms
        hp.write_alm(str(smica_dedopplered_noise_dopp_sims)+'doppler_boosted_alm_symmmasked_w_smica_dDD_noise_beam_deconvolved_dir_'+str(n+1)+'_sim_'+str(i+1)+'.fits',doppler_noise_symmmasked_alm_dd,overwrite=True)
        hp.write_alm(str(smica_dedopplered_noise_ab_sims)+'ab_boosted_alm_symmmasked_w_smica_dDD_noise_beam_deconvolved_dir_'+str(n+1)+'_sim_'+str(i+1)+'.fits',aberration_noise_symmmasked_alm_dd,overwrite=True)
        hp.write_alm(str(smica_dedopplered_noise_abdopp_sims)+'abdopp_boosted_alm_symmmasked_w_smica_dDD_noise_beam_deconvolved_dir_'+str(n+1)+'_sim_'+str(i+1)+'.fits',abdopp_noise_symmmasked_alm_dd,overwrite=True)
        
        end_sim = time.time()
        print(end_sim-start_sim)
        
    end_dir = time.time()
    print(end_dir-start_dir)
        
pool = mp.Pool(processes=nprocesses)
pool.map(noise_nilc_loop, range(hp.nside2npix(nside_rot)))
pool.terminate()
pool.join()

# ESTIMATOR

## Main Pipeline

### NILC

In [ ]:
############################################
## Estimator: Mask and Deconvolved Noise ###
############################################
################## NILC ####################
############################################
########### MASK: UT78symmApod10 ###########
############################################

nilc_dd_w_noise_sims_ab_sims = '/nilc_dd_ab_w_noise_sims/'
nilc_dd_w_noise_sims_dopp_sims = '/nilc_dd_dopp_w_noise_sims/'
nilc_dd_w_noise_sims_abdopp_sims = '/nilc_dd_abdopp_w_noise_sims/'
nilc_dd_w_noise_sims_ab_results =  '/nilc_dd_ab_w_noise_results/'
nilc_dd_w_noise_sims_dopp_results = '/nilc_dd_dopp_w_noise_results/'
nilc_dd_w_noise_sims_abdopp_results = '/nilc_dd_abdopp_w_noise_results/'

nside_rot = 2 # setting the directions do you want to apply the modulations

nthreads= 4
c_nthreads_limit(nthreads)
nprocesses = 3

clfile='cl_TT_planck_2019.fits'
cl = hp.read_cl(clfile)
htheofast(cl,mastermatrixfile='new_M-ell-ellp-U79symm-apod10-maxdl2500-lmax2500.csv',nlfile='/Noise_analysis/nl_nilc_T_symmmasked.fits',binsize=10,plot_std=True,threads=8)#htheofast

def noise_loop_dd(n):
    start = time.time()
    for i in range(64):
        print(n,i)
        doppler_alm_dd = hp.read_alm(nilc_dd_w_noise_sims_dopp_sims+'doppler_boosted_alm_symmmasked_w_nilc_dd_and_noise_beam_deconvolved_dir_'+str(n+1)+'_sim_'+str(i+1)+'.fits')
        aberration_alm_dd = hp.read_alm(nilc_dd_w_noise_sims_ab_sims+'ab_boosted_alm_symmmasked_w_nilc_dd_and_noise_beam_deconvolved_dir_'+str(n+1)+'_sim_'+str(i+1)+'.fits')        
        abdopp_alm_dd = hp.read_alm(nilc_dd_w_noise_sims_abdopp_sims+'abdopp_boosted_alm_symmmasked_w_nilc_dd_and_noise_beam_deconvolved_dir_'+str(n+1)+'_sim_'+str(i+1)+'.fits')        
        
        # Reordering from healpy to healpix fortran index order.
        doppler_alm_dd = reorder_idxpy2pix(doppler_alm_dd,threads=nthreads)
        aberration_alm_dd = reorder_idxpy2pix(aberration_alm_dd,threads=nthreads)
        abdopp_alm_dd = reorder_idxpy2pix(abdopp_alm_dd,threads=nthreads)
        
        # Estimating beta^A,beta^D and beta^B
        betafast(aberration_alm_dd,binsize=10,threads=nthreads,sufixname=nilc_dd_w_noise_sims_ab_results+'nilc_symmmasked_w_noise_and_dd_binsize10_ab_dir_'+str(n+1)+'_sim_'+str(i+1)+'_',verbose=False,export_results=True);
        betafast(doppler_alm_dd,binsize=10,threads=nthreads,sufixname=nilc_dd_w_noise_sims_dopp_results+'nilc_symmmasked_w_noise_and_dd_binsize10_dopp_dir_'+str(n+1)+'_sim_'+str(i+1)+'_',verbose=False,export_results=True);
        betafast(abdopp_alm_dd,binsize=10,threads=nthreads,sufixname=nilc_dd_w_noise_sims_abdopp_results+'nilc_symmmasked_w_noise_and_dd_binsize10_abdopp_dir_'+str(n+1)+'_sim_'+str(i+1)+'_',verbose=False,export_results=True);
    end = time.time()
    print(end-start)
        
pool = mp.Pool(processes=nprocesses)
pool.map(noise_loop_dd, range(hp.nside2npix(nside_rot)))
pool.terminate()
pool.join()

### SMICA

In [ ]:
############################################
## Estimator: Mask and Deconvolved Noise ###
############################################
################## SMICA ###################
############################################
########### MASK: UT78symmApod10 ###########
############################################

smica_dd_w_noise_sims_ab_sims = '' 
smica_dd_w_noise_sims_dopp_sims = ''
smica_dd_w_noise_sims_abdopp_sims = '' 
smica_dd_w_noise_sims_ab_results = '' 
smica_dd_w_noise_sims_dopp_results = ''
smica_dd_w_noise_sims_abdopp_results = ''

nside_rot = 2 # setting the directions do you want to apply the modulations

clfile='cl_TT_planck_2019.fits'
cl = hp.read_cl(clfile)
#,mastermatrixfile='/home/pedro/hp_nvme/ssd_extra/mask_files/M-ell-ellp-U79symm-apod10-maxdl2500-lmax2500.csv'
htheofast(cl,mastermatrixfile='new_M-ell-ellp-U79symm-apod10-maxdl2500-lmax2500.csv',nlfile='nilc_nl_symmmasked.fits',binsize=10,plot_std=True,threads=8)#htheofast

def noise_loop_dd(n):
    start = time.time()
    for i in range(64):
        print(n,i)
        doppler_alm_dd = hp.read_alm(nilc_dd_w_noise_sims_dir1+'doppler_boosted_alm_symmmasked_w_nilc_dd_and_noise_beam_deconvolved_dir_'+str(n+1)+'_sim_'+str(i+1)+'.fits')
        aberration_alm_dd = hp.read_alm(nilc_dd_w_noise_sims_dir2+'ab_boosted_alm_symmmasked_w_nilc_dd_and_noise_beam_deconvolved_dir_'+str(n+1)+'_sim_'+str(i+1)+'.fits')        
        abdopp_alm_dd = hp.read_alm(nilc_dd_w_noise_sims_dir3+'abdopp_boosted_alm_symmmasked_w_nilc_dd_and_noise_beam_deconvolved_dir_'+str(n+1)+'_sim_'+str(i+1)+'.fits')        
        
        # Reordering from healpy to healpix fortran index order.
        doppler_alm_dd = reorder_idxpy2pix(doppler_alm_dd,threads=8)
        aberration_alm_dd = reorder_idxpy2pix(aberration_alm_dd,threads=8)
        abdopp_alm_dd = reorder_idxpy2pix(abdopp_alm_dd,threads=8)
        
        # Estimating beta^A,beta^D and beta^B
        betafast(aberration_alm_dd,binsize=10,threads=4,sufixname=nilc_dd_w_noise_sims_dir2+'nilc_symmmasked_w_noise_and_dd_binsize10_ab_dir_'+str(n+1)+'_sim_'+str(i+1)+'_',verbose=False,export_results=True);
        betafast(doppler_alm_dd,binsize=10,threads=4,sufixname=nilc_dd_w_noise_sims_dir1+'nilc_symmmasked_w_noise_and_dd_binsize10_dopp_dir_'+str(n+1)+'_sim_'+str(i+1)+'_',verbose=False,export_results=True);
        betafast(abdopp_alm_dd,binsize=10,threads=4,sufixname=nilc_dd_w_noise_sims_dir3+'nilc_symmmasked_w_noise_and_dd_binsize10_abdopp_dir_'+str(n+1)+'_sim_'+str(i+1)+'_',verbose=False,export_results=True);
    end = time.time()
    print(end-start)
        
pool = mp.Pool(processes=3)
pool.map(noise_loop_dd, range(hp.nside2npix(nside_rot)))
pool.terminate()
pool.join()

## Estimator - Cross-Check Pipeline

### NILC (CCP)

In [ ]:
############################################
## Estimator: Mask and dDD Noise ###########
############################################
################## NILC ####################
############################################
########### MASK: UT78symmApod10 ###########
############################################

nilc_dedopplered_noise_ab_sims = '' 
nilc_dedopplered_noise_dopp_sims = ''
nilc_dedopplered_noise_abdopp_sims = ''
nilc_dedopplered_noise_ab_results = '' 
nilc_dedopplered_noise_dopp_results = ''
nilc_dedopplered_noise_abdopp_results = ''

nside_rot = 2 # setting the directions do you want to apply the modulations

clfile='cl_TT_planck_2019.fits'
cl = hp.read_cl(clfile)
#,mastermatrixfile='/home/pedro/hp_nvme/ssd_extra/mask_files/M-ell-ellp-U79symm-apod10-maxdl2500-lmax2500.csv'
htheofast(cl,mastermatrixfile='new_M-ell-ellp-U79symm-apod10-maxdl2500-lmax2500.csv',nlfile='nilc_nl_symmmasked.fits',binsize=10,plot_std=True,threads=8)#htheofast

def noise_loop_dd(n):
    start = time.time()
    for i in range(64):
        print(n,i)
        doppler_alm_dd = hp.read_alm(nilc_dedopplered_noise_dopp_sims+'doppler_boosted_alm_symmmasked_w_nilc_dDD_noise_beam_deconvolved_dir_'+str(n+1)+'_sim_'+str(i+1)+'.fits')
        aberration_alm_dd = hp.read_alm(nilc_dedopplered_noise_ab_sims+'ab_boosted_alm_symmmasked_w_nilc_dDD_noise_beam_deconvolved_dir_'+str(n+1)+'_sim_'+str(i+1)+'.fits')        
        abdopp_alm_dd = hp.read_alm(nilc_dedopplered_noise_abdopp_sims+'abdopp_boosted_alm_symmmasked_w_nilc_dDD_noise_beam_deconvolved_dir_'+str(n+1)+'_sim_'+str(i+1)+'.fits')        
        
        doppler_alm_dd = reorder_idxpy2pix(doppler_alm_dd,threads=8)
        aberration_alm_dd = reorder_idxpy2pix(aberration_alm_dd,threads=8)
        abdopp_alm_dd = reorder_idxpy2pix(abdopp_alm_dd,threads=8)
        
        betafast(aberration_alm_dd,binsize=10,threads=4,sufixname=nilc_dedopplered_noise_ab_results+'nilc_symmmasked_noise_dedopplered_binsize10_ab_dir_'+str(n+1)+'_sim_'+str(i+1)+'_',verbose=False,export_results=True);
        betafast(doppler_alm_dd,binsize=10,threads=4,sufixname=nilc_dedopplered_noise_dopp_results+'nilc_symmmasked_noise_dedopplered_binsize10_dopp_dir_'+str(n+1)+'_sim_'+str(i+1)+'_',verbose=False,export_results=True);
        betafast(abdopp_alm_dd,binsize=10,threads=4,sufixname=nilc_dedopplered_noise_abdopp_results+'nilc_symmmasked_noise_dedopplered_binsize10_abdopp_dir_'+str(n+1)+'_sim_'+str(i+1)+'_',verbose=False,export_results=True);
    end = time.time()
    print(end-start)
        
pool = mp.Pool(processes=3)
pool.map(noise_loop_dd, range(hp.nside2npix(nside_rot)))
pool.terminate()
pool.join()

### SMICA (CCP)

In [ ]:
############################################
## Estimator: Mask and dDD Noise ###########
############################################
################## SMICA ###################
############################################
########### MASK: UT78symmApod10 ###########
############################################

smica_dedopplered_noise_ab_sims = '' 
smica_dedopplered_noise_dopp_sims = ''
smica_dedopplered_noise_abdopp_sims = ''
smica_dedopplered_noise_ab_results = '' 
smica_dedopplered_noise_dopp_results = ''
smica_dedopplered_noise_abdopp_results = ''

nside_rot = 2 # setting the directions do you want to apply the modulations

clfile='cl_TT_planck_2019.fits'
cl = hp.read_cl(clfile)
#,mastermatrixfile='/home/pedro/hp_nvme/ssd_extra/mask_files/M-ell-ellp-U79symm-apod10-maxdl2500-lmax2500.csv'
htheofast(cl,mastermatrixfile='new_M-ell-ellp-U79symm-apod10-maxdl2500-lmax2500.csv',nlfile='nilc_nl_symmmasked.fits',binsize=10,plot_std=True,threads=8)#htheofast

def noise_loop_dd(n):
    start = time.time()
    for i in range(64):
        print(n,i)
        doppler_alm_dd = hp.read_alm(smica_dedopplered_noise_dopp_sims+'doppler_boosted_alm_symmmasked_w_nilc_dDD_noise_beam_deconvolved_dir_'+str(n+1)+'_sim_'+str(i+1)+'.fits')
        aberration_alm_dd = hp.read_alm(smica_dedopplered_noise_ab_sims+'ab_boosted_alm_symmmasked_w_nilc_dDD_noise_beam_deconvolved_dir_'+str(n+1)+'_sim_'+str(i+1)+'.fits')        
        abdopp_alm_dd = hp.read_alm(smica_dedopplered_noise_abdopp_sims+'abdopp_boosted_alm_symmmasked_w_nilc_dDD_noise_beam_deconvolved_dir_'+str(n+1)+'_sim_'+str(i+1)+'.fits')        
        
        doppler_alm_dd = reorder_idxpy2pix(doppler_alm_dd,threads=8)
        aberration_alm_dd = reorder_idxpy2pix(aberration_alm_dd,threads=8)
        abdopp_alm_dd = reorder_idxpy2pix(abdopp_alm_dd,threads=8)
        
        betafast(aberration_alm_dd,binsize=10,threads=4,sufixname=smica_dedopplered_noise_ab_results+'nilc_symmmasked_noise_dedopplered_binsize10_ab_dir_'+str(n+1)+'_sim_'+str(i+1)+'_',verbose=False,export_results=True);
        betafast(doppler_alm_dd,binsize=10,threads=4,sufixname=smica_dedopplered_noise_dopp_results+'nilc_symmmasked_noise_dedopplered_binsize10_dopp_dir_'+str(n+1)+'_sim_'+str(i+1)+'_',verbose=False,export_results=True);
        betafast(abdopp_alm_dd,binsize=10,threads=4,sufixname=smica_dedopplered_noise_abdopp_results+'nilc_symmmasked_noise_dedopplered_binsize10_abdopp_dir_'+str(n+1)+'_sim_'+str(i+1)+'_',verbose=False,export_results=True);
    end = time.time()
    print(end-start)
        
pool = mp.Pool(processes=3)
pool.map(noise_loop_dd, range(hp.nside2npix(nside_rot)))
pool.terminate()
pool.join()

# Examples with EE

## EE null and only noise

In [ ]:
## Only noise
##########################################
## Applying Mask and Deconvolved Noise ###
##########################################
######## Gaussian SMICA ##################
##########################################


mask_files_dir = '/mask_files/'
noise_files_dir = '/noise_T_E_B/'

nsims = 3072

nthreads= 4
c_nthreads_limit(nthreads)
nprocesses = 3

results_dir = '/EE_null_and_only_noise_results/'

nside_rot = 2 # setting the directions do you want to apply the modulations

clTT = hp.read_cl("cl_TT_planck_2019.fits")
clTE = hp.read_cl("cl_TE_planck_2019.fits")
clTB = np.zeros(clTT.size)
clEE = hp.read_cl("cl_EE_planck_2019.fits")
clEB = np.zeros(clTT.size)
clBB = np.zeros(clTT.size)

htheofast(clEE,lmax=1448,mastermatrixfile='new_M-ell-ellp-U79symm-apod10-maxdl2500-lmax2500.csv',nlfile='smica_nl_E_correct_symmmasked.fits',binsize=10,plot_std=False,threads=8)#htheofast


symmmask = hp.read_map(str(mask_files_dir)+'UT79symmapod10.fits',verbose=False)
def noise_nilc_loop(i):
    start_sim = time.time()
    print(i+1)
    noise_alm_E = hp.read_alm(str(noise_files_dir)+'debeamed_smica_E_noise_'+str(np.random.randint(300))+'.fits')
    noise_alm_T = np.zeros(noise_alm_E.size,dtype=np.complex128)
    noise_alm_B = np.zeros(noise_alm_E.size,dtype=np.complex128)
    noise_I,noise_Q,noise_U = hp.alm2map([noise_alm_T,noise_alm_E,noise_alm_B],nside=2048)

    #generating maps and adding noise
    map_I, map_Q, map_U = hp.synfast((clTT, clTE, clTB, clEE, clEB, clBB),nside=2048,lmax=1450,new=False)    

    map_I = map_I + noise_I
    map_Q = map_Q + noise_Q
    map_U = map_U + noise_U

    #applying symm mask
    map_I = map_I*symmmask
    map_Q = map_Q*symmmask
    map_U = map_U*symmmask

    #generating symmmasked alms
    alm_T,alm_E,alm_B = hp.map2alm([map_I,map_Q,map_U],lmax=1450,iter=1)
    alm_E = reorder_idxpy2pix(alm_E)
    #saving alms
#    hp.write_alm('/home/hdd3/cmb_boost_tmp/null_sims_2/beta_null_alm_symmmasked_w_smica_dd_and_noise_beam_deconvolved_sim_'+str(i+1)+'.fits',abdopp_noise_symmmasked_alm_dd,overwrite=True)
    betafast(alm_E,binsize=10,lmax=1448,threads=8,sufixname=results_dir+'smica_EE_symmmasked_only_noise_binsize10_sim_'+str(i+1)+'_',verbose=False,export_results=True);

    end_sim = time.time()
    print(end_sim-start_sim)

        
pool = mp.Pool(processes=nprocesses)
pool.map(noise_nilc_loop, range(nsims))
pool.terminate()
pool.join()

In [ ]:
## Only noise
##########################################
## Applying Mask and Deconvolved Noise ###
##########################################
######## Gaussian SMICA ##################
##########################################


mask_files_dir = '/mask_files/'
noise_files_dir = '/noise_T_E_B/'

nsims = 1024

nthreads= 4
c_nthreads_limit(nthreads)
nprocesses = 3

results_dir = '/EE_null_and_only_noise_results/'

nside_rot = 2 # setting the directions do you want to apply the modulations

clTT = hp.read_cl("cl_TT_planck_2019.fits")
clTE = hp.read_cl("cl_TE_planck_2019.fits")
clTB = np.zeros(clTT.size)
clEE = hp.read_cl("cl_EE_planck_2019.fits")
clEB = np.zeros(clTT.size)
clBB = np.zeros(clTT.size)

htheofast(clEE,lmax=1448,mastermatrixfile='new_M-ell-ellp-U79symm-apod10-maxdl2500-lmax2500.csv',nlfile='nilc_nl_E_correct_symmmasked.fits',binsize=10,plot_std=False,threads=8)#htheofast


symmmask = hp.read_map(str(mask_files_dir)+'UT79symmapod10.fits',verbose=False)
def noise_nilc_loop(i):
    start_sim = time.time()
    print(i+1)
    noise_alm_E = hp.read_alm(str(noise_files_dir)+'debeamed_nilc_E_noise_'+str(np.random.randint(300))+'.fits')
    noise_alm_T = np.zeros(noise_alm_E.size,dtype=np.complex128)
    noise_alm_B = np.zeros(noise_alm_E.size,dtype=np.complex128)
    noise_I,noise_Q,noise_U = hp.alm2map([noise_alm_T,noise_alm_E,noise_alm_B],nside=2048)

    #generating maps and adding noise
    map_I, map_Q, map_U = hp.synfast((clTT, clTE, clTB, clEE, clEB, clBB),nside=2048,lmax=1450,new=False)    

    map_I = map_I + noise_I
    map_Q = map_Q + noise_Q
    map_U = map_U + noise_U

    #applying symm mask
    map_I = map_I*symmmask
    map_Q = map_Q*symmmask
    map_U = map_U*symmmask

    #generating symmmasked alms
    alm_T,alm_E,alm_B = hp.map2alm([map_I,map_Q,map_U],lmax=1450,iter=1)
    alm_E = reorder_idxpy2pix(alm_E)
    #saving alms
#    hp.write_alm('/home/hdd3/cmb_boost_tmp/null_sims_2/beta_null_alm_symmmasked_w_smica_dd_and_noise_beam_deconvolved_sim_'+str(i+1)+'.fits',abdopp_noise_symmmasked_alm_dd,overwrite=True)
    betafast(alm_E,binsize=10,lmax=1448,threads=8,sufixname=results_dir+'nilc_EE_symmmasked_only_noise_binsize10_sim_'+str(i+1)+'_',verbose=False,export_results=True);

    end_sim = time.time()
    print(end_sim-start_sim)

        
pool = mp.Pool(processes=nprocesses)
pool.map(noise_nilc_loop, range(nsims))
pool.terminate()
pool.join()

## EE simulations (MP) - Example

In [ ]:
##########################################
## Applying Mask and Deconvolved Noise ###
##########################################
############### SMICA EE #################
##########################################

lmax_var = 1450

ideal_abdopp_sims_dir = '/home/pedro/wd_red_1tb/cmb_boost_legacy_v2_wd_red_1tb/ab_on_polarization/milliways/'
ideal_dopp_sims_dir = '/home/pedro/wd_red_1tb/dopp_ee_sims/'

mask_files_dir = ''
noise_files_dir = '/noise_T_E_B/'
abdopp_ee_dir = '/ab_on_polarization/'
nside_rot = 2 # setting the directions do you want to apply the modulations
nsims_per_dir = 64

nthreads= 8
c_nthreads_limit(nthreads)

alm_T = np.zeros(hp.Alm.getsize(lmax=lmax_var),dtype=np.complex128)
alm_B = alm_T
noise_alm_T = np.zeros(hp.Alm.getsize(lmax=lmax_var),dtype=np.complex128)
noise_alm_B = noise_alm_T

py2pix_r = reorder_idx_idxpy2pix(lmax=1450) # function to get healpy to healpix index reorder array
pix2py_r = reorder_idx_idxpix2py(lmax=1450) # function to get healpix to healpy index reorder array

symmmask = hp.read_map(mask_files_dir+'mask_pol_symm.fits')
def noise_smica_loop(n):
    start_dir = time.time()
    for i in range(nsims_per_dir):
        start_sim = time.time()
        print(n,i+1)
        noise_alm_E = hp.read_alm('debeamed_dd_removed_smica_E_noise_'+str(np.random.randint(300))+'.fits')
        noise_alm_E = noise_alm_E[py2pix_r] # healpy to healpix index reorder
        noise_alm_E = noise_alm_E[:hp.Alm.getsize(lmax=lmax_var)]
        noise_alm_E = noise_alm_E[pix2py_r] # healpix to healpy index reorder

        noise_I,noise_Q,noise_U = hp.alm2map([noise_alm_T,noise_alm_E,noise_alm_B],nside=2048)
        
        #importing alms w DD
        ab_alm_dd_E = hp.read_alm(ideal_abdopp_sims_dir+'almE_ab_rot_dir_'+str(n+1)+'_sim_'+str(i+1)+'.fits')
        dopp_alm_dd_E = hp.read_alm(ideal_dopp_sims_dir+'dopp_EE_boosted_alm_dir_'+str(n+1)+'_sim_'+str(i+1)+'.fits')
        abdopp_alm_dd_E = hp.read_alm(ideal_abdopp_sims_dir+'abdopp_EE_boosted_alm_dir_'+str(n+1)+'_sim_'+str(i+1)+'.fits')
        
        #generating maps and adding noise

        ab_map_dd_I,ab_map_dd_Q,ab_map_dd_U = hp.alm2map([alm_T,ab_alm_dd_E,alm_B],nside=2048,verbose=False)
        dopp_map_dd_I,dopp_map_dd_Q,dopp_map_dd_U = hp.alm2map([alm_T,dopp_alm_dd_E,alm_B],nside=2048,verbose=False)
        abdopp_map_dd_I,abdopp_map_dd_Q,abdopp_map_dd_U = hp.alm2map([alm_T,abdopp_alm_dd_E,alm_B],nside=2048,verbose=False)

        ab_noise_map_dd_I = ab_map_dd_I + noise_I
        ab_noise_map_dd_Q = ab_map_dd_Q + noise_Q
        ab_noise_map_dd_U = ab_map_dd_U + noise_U        
        
        dopp_noise_map_dd_I = dopp_map_dd_I + noise_I
        dopp_noise_map_dd_Q = dopp_map_dd_Q + noise_Q
        dopp_noise_map_dd_U = dopp_map_dd_U + noise_U        
        
        abdopp_noise_map_dd_I = abdopp_map_dd_I + noise_I
        abdopp_noise_map_dd_Q = abdopp_map_dd_Q + noise_Q
        abdopp_noise_map_dd_U = abdopp_map_dd_U + noise_U
        
        #applying symm mask
        ab_noise_masked_map_dd_I = ab_noise_map_dd_I*symmmask
        ab_noise_masked_map_dd_Q = ab_noise_map_dd_Q*symmmask
        ab_noise_masked_map_dd_U = ab_noise_map_dd_U*symmmask
        
        dopp_noise_masked_map_dd_I = dopp_noise_map_dd_I*symmmask
        dopp_noise_masked_map_dd_Q = dopp_noise_map_dd_Q*symmmask
        dopp_noise_masked_map_dd_U = dopp_noise_map_dd_U*symmmask
        
        abdopp_noise_masked_map_dd_I = abdopp_noise_map_dd_I*symmmask
        abdopp_noise_masked_map_dd_Q = abdopp_noise_map_dd_Q*symmmask
        abdopp_noise_masked_map_dd_U = abdopp_noise_map_dd_U*symmmask
                
        #generating symmmasked alms
        ab_noise_symmmasked_dd_alm_T, ab_noise_symmmasked_dd_alm_E, ab_noise_symmmasked_dd_alm_B = hp.map2alm([ab_noise_masked_map_dd_I,ab_noise_masked_map_dd_Q,ab_noise_masked_map_dd_U],lmax=lmax_var,iter=1)
        dopp_noise_symmmasked_dd_alm_T, dopp_noise_symmmasked_dd_alm_E, dopp_noise_symmmasked_dd_alm_B = hp.map2alm([dopp_noise_masked_map_dd_I,dopp_noise_masked_map_dd_Q,dopp_noise_masked_map_dd_U],lmax=lmax_var,iter=1)
        abdopp_noise_symmmasked_dd_alm_T, abdopp_noise_symmmasked_dd_alm_E, abdopp_noise_symmmasked_dd_alm_B = hp.map2alm([abdopp_noise_masked_map_dd_I,abdopp_noise_masked_map_dd_Q,abdopp_noise_masked_map_dd_U],lmax=lmax_var,iter=1)
        
        #saving alms
        hp.write_alm('/home/pedro/wd_red_1tb/ee_dD/ab_boosted_alm_EE_v2_symmmasked_w_smica_dD_noise_beam_deconvolved_dir_'+str(n+1)+'_sim_'+str(i+1)+'.fits',ab_noise_symmmasked_dd_alm_E,overwrite=True)
        hp.write_alm('/home/pedro/wd_red_1tb/ee_dD/dopp_boosted_alm_EE_v2_symmmasked_w_smica_dD_noise_beam_deconvolved_dir_'+str(n+1)+'_sim_'+str(i+1)+'.fits',dopp_noise_symmmasked_dd_alm_E,overwrite=True)
        hp.write_alm('/home/pedro/wd_red_1tb/ee_dD/abdopp_boosted_alm_EE_v2_symmmasked_w_smica_dD_noise_beam_deconvolved_dir_'+str(n+1)+'_sim_'+str(i+1)+'.fits',abdopp_noise_symmmasked_dd_alm_E,overwrite=True)
    
        end_sim = time.time()
        print(end_sim-start_sim)
        
    end_dir = time.time()
    print(end_dir-start_dir)
        
for n in tqdm(range(48)):
    noise_smica_loop(n)

## Estimating deDoplered EE (CCP) - Example

### NILC EE (CCP)

In [ ]:
nilc_dedopplered_results = '/ee_dd_results/'
nside_rot = 2 # setting the directions do you want to apply the modulations

clfile='cl_EE_planck_2019.fits'
cl = hp.read_cl(clfile)
#,mastermatrixfile='/home/pedro/hp_nvme/ssd_extra/mask_files/M-ell-ellp-U79symm-apod10-maxdl2500-lmax2500.csv'
htheofast(cl,lmax=1448,mastermatrixfile='master_matrix_EE.csv',nlfile='nilc_nl_E_correct_symmmasked.fits',binsize=10,threads=8)#htheofast

def estimator_loop_deD(n):
    start = time.time()
    for i in range(64):
        print(n,i)
        doppler_alm_dd = hp.read_alm('dopp_boosted_alm_EE_v2_symmmasked_w_nilc_dD_noise_beam_deconvolved_dir_'+str(n+1)+'_sim_'+str(i+1)+'.fits')
        aberration_alm_dd = hp.read_alm('ab_boosted_alm_EE_v2_symmmasked_w_nilc_dD_noise_beam_deconvolved_dir_'+str(n+1)+'_sim_'+str(i+1)+'.fits')        
        abdopp_alm_dd = hp.read_alm('abdopp_boosted_alm_EE_v2_symmmasked_w_nilc_dD_noise_beam_deconvolved_dir_'+str(n+1)+'_sim_'+str(i+1)+'.fits')        
        
        doppler_alm_dd = reorder_idxpy2pix(doppler_alm_dd,threads=8)
        aberration_alm_dd = reorder_idxpy2pix(aberration_alm_dd,threads=8)
        abdopp_alm_dd = reorder_idxpy2pix(abdopp_alm_dd,threads=8)
        
        betafast(aberration_alm_dd,lmax=1448,binsize=10,threads=4,sufixname=nilc_dedopplered_results+'nilc_EE_symmmasked_noise_dedopplered_binsize10_ab_dir_'+str(n+1)+'_sim_'+str(i+1)+'_',verbose=False,export_results=True);
        betafast(doppler_alm_dd,lmax=1448,binsize=10,threads=4,sufixname=nilc_dedopplered_results+'nilc_EE_symmmasked_noise_dedopplered_binsize10_dopp_dir_'+str(n+1)+'_sim_'+str(i+1)+'_',verbose=False,export_results=True);
        betafast(abdopp_alm_dd,lmax=1448,binsize=10,threads=4,sufixname=nilc_dedopplered_results+'nilc_EE_symmmasked_noise_dedopplered_binsize10_abdopp_dir_'+str(n+1)+'_sim_'+str(i+1)+'_',verbose=False,export_results=True);
    end = time.time()
    print(end-start)
        
pool = mp.Pool(processes=3)
pool.map(estimator_loop_deD, range(hp.nside2npix(nside_rot)))
pool.terminate()
pool.join()

### SMICA EE (CCP)

In [ ]:
nilc_dedopplered_results = '/home/pedro/hp_nvme/cmb_boost_legacy_v2/EE_data/ee_dd_results/'
nside_rot = 2 # setting the directions do you want to apply the modulations

clfile='cl_EE_planck_2019.fits'
cl = hp.read_cl(clfile)
#,mastermatrixfile='/home/pedro/hp_nvme/ssd_extra/mask_files/M-ell-ellp-U79symm-apod10-maxdl2500-lmax2500.csv'
htheofast(cl,lmax=1448,mastermatrixfile='/home/pedro/hp_nvme/cmb_boost_legacy_v2/Pol_mask/master_matrix_EE.csv',nlfile='/home/pedro/hp_nvme/cmb_boost_legacy_v2/EE_data/smica_nl_E_correct_symmmasked.fits',binsize=10,threads=8)#htheofast

def estimator_loop_deD(n):
    start = time.time()
    for i in range(64):
        print(n,i)
        doppler_alm_dd = hp.read_alm('/home/pedro/hp_nvme/cmb_boost_legacy_v2/EE_data/EE_dD_alms/dopp_boosted_alm_EE_v2_symmmasked_w_smica_dD_noise_beam_deconvolved_dir_'+str(n+1)+'_sim_'+str(i+1)+'.fits')
        aberration_alm_dd = hp.read_alm('/home/pedro/hp_nvme/cmb_boost_legacy_v2/EE_data/EE_dD_alms/ab_boosted_alm_EE_v2_symmmasked_w_smica_dD_noise_beam_deconvolved_dir_'+str(n+1)+'_sim_'+str(i+1)+'.fits')        
        abdopp_alm_dd = hp.read_alm('/home/pedro/hp_nvme/cmb_boost_legacy_v2/EE_data/EE_dD_alms/abdopp_boosted_alm_EE_v2_symmmasked_w_smica_dD_noise_beam_deconvolved_dir_'+str(n+1)+'_sim_'+str(i+1)+'.fits')        
        
        doppler_alm_dd = reorder_idxpy2pix(doppler_alm_dd,threads=8)
        aberration_alm_dd = reorder_idxpy2pix(aberration_alm_dd,threads=8)
        abdopp_alm_dd = reorder_idxpy2pix(abdopp_alm_dd,threads=8)
        
        betafast(aberration_alm_dd,lmax=1448,binsize=10,threads=4,sufixname=nilc_dedopplered_results+'smica_EE_symmmasked_noise_dedopplered_binsize10_ab_dir_'+str(n+1)+'_sim_'+str(i+1)+'_',verbose=False,export_results=True);
        betafast(doppler_alm_dd,lmax=1448,binsize=10,threads=4,sufixname=nilc_dedopplered_results+'smica_EE_symmmasked_noise_dedopplered_binsize10_dopp_dir_'+str(n+1)+'_sim_'+str(i+1)+'_',verbose=False,export_results=True);
        betafast(abdopp_alm_dd,lmax=1448,binsize=10,threads=4,sufixname=nilc_dedopplered_results+'smica_EE_symmmasked_noise_dedopplered_binsize10_abdopp_dir_'+str(n+1)+'_sim_'+str(i+1)+'_',verbose=False,export_results=True);
    end = time.time()
    print(end-start)
        
pool = mp.Pool(processes=3)
pool.map(estimator_loop_deD, range(hp.nside2npix(nside_rot)))
pool.terminate()
pool.join()